# Various Tests of Marlowe Runtime

## Preliminaries

Record version numbers

In [1]:
marlowe-cli --version

marlowe-cli 0.0.9.0 «mainnet»


In [2]:
marlowe --version

Invalid option `--version'

Usage: marlowe [--history-host HOST_NAME] [--history-command-port PORT_NUMBER] 
               [--history-query-port PORT_NUMBER] 
               [--history-sync-port PORT_NUMBER] [--discovery-host HOST_NAME] 
               [--discovery-query-port PORT_NUMBER] [--tx-host HOST_NAME] 
               [--tx-command-port PORT_NUMBER] (COMMAND | COMMAND | COMMAND)

  Command line interface for managing Marlowe smart contracts on Cardano.


: 1

<span style="color: red; font-weight: bold">It looks like <code>marlowe</code> doesn't support <code>--version</code>.</span>

In [3]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [4]:
git rev-parse HEAD

975627a11890a7121d030ec8236c11469febd080


## Set connections to Marlowe Runtime services

In [5]:
export MARLOWE_RT_HISTORY_HOST="127.0.0.1"
export MARLOWE_RT_HISTORY_COMMAND_PORT=23717
export MARLOWE_RT_HISTORY_QUERY_PORT=23718
export MARLOWE_RT_HISTORY_SYNC_PORT=23719

export MARLOWE_RT_DISCOVERY_HOST="127.0.0.1"
export MARLOWE_RT_DISCOVERY_QUERY_PORT=23721

export MARLOWE_RT_TX_HOST="127.0.0.1"
export MARLOWE_RT_TX_COMMAND_PORT=23723

### Setup the faucet.

Set the location of keys.

In [6]:
TREASURY=/extra/iohk/networks/treasury

Set the faucet.

In [7]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDR"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Select network

In [8]:
MAGIC=(--testnet-magic 2)
export CARDANO_TESTNET_MAGIC=2
export CARDANO_NODE_SOCKET_PATH=/extra/iohk/networks/preview/node.socket

### Time computations

In [9]:
SECOND=1000
MINUTE=$((60 * SECOND))
HOUR=$((60 * MINUTE))
DAY=$((24 * HOUR))

In [10]:
NOW="$(($(date -u +%s) * SECOND))"
echo "$NOW"

1665433958000


### Check that the reference script has been published

Check that the Marlowe semantics validator was published.

In [11]:
marlowe-cli transaction find-published


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#2"
    }
}


### Participants

#### The Party

In [12]:
PARTY_PREFIX="$TREASURY/john-fletcher"
PARTY_SKEY="$PARTY_PREFIX".skey
PARTY_VKEY="$PARTY_PREFIX".vkey

Create the first party's keys, if necessary.

In [13]:
if [[ ! -e "$PARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_SKEY"      \
                              --verification-key-file "$PARTY_VKEY"
fi
PARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PARTY_VKEY")
echo "$PARTY_ADDR"

addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q


Fund the address.

In [14]:
marlowe-cli util faucet --out-file /dev/null \
                        --submit 600 \
                        --lovelace 250000000 \
                        --faucet-address "$FAUCET_ADDR" \
                        --required-signer "$FAUCET_SKEY" \
                        "$PARTY_ADDR"

83dfb2998a5de85d0696484cd098eb39ba002b18670ca4e29364deb4029dbc54


#### The Counterparty

In [15]:
COUNTERPARTY_PREFIX="$TREASURY/thomas-kyd"
COUNTERPARTY_SKEY="$COUNTERPARTY_PREFIX".skey
COUNTERPARTY_VKEY="$COUNTERPARTY_PREFIX".vkey

Create the second party's keys, if necessary.

In [16]:
if [[ ! -e "$COUNTERPARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$COUNTERPARTY_SKEY"      \
                              --verification-key-file "$COUNTERPARTY_VKEY"
fi
COUNTERPARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$COUNTERPARTY_VKEY")
echo "$COUNTERPARTY_ADDR"

addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5


Fund the address.

In [17]:
marlowe-cli util faucet --out-file /dev/null \
                        --submit 600  \
                        --lovelace 250000000 \
                        --faucet-address "$FAUCET_ADDR" \
                        --required-signer "$FAUCET_SKEY" \
                        "$COUNTERPARTY_ADDR"

c26d7d4d96732316b186ef983338bfa9809ae979edf08484999d3ba50386a389


## The Contract

We set the parameters for the ACTUS PAM contract.

In [18]:
MINIMUM_ADA=3000000

FIXED_POINT=1000000
PRINCIPAL=100
INTEREST_RATE=0.02
INTEREST=$(jq -n $PRINCIPAL*$INTEREST_RATE)

STATUS_DATE=$(date -d "$(date -u -R -d @$((NOW/1000)))" +"%Y-%m-%dT00:00:00")
INITIAL_EXCHANGE_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 1 year" +"%Y-01-01T00:00:00")
MATURITY_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 2 year" +"%Y-01-01T00:00:00")

LENDING_DEADLINE=$((NOW+12*HOUR))
REPAYMENT_DEADLINE=$((NOW+24*HOUR))

In [19]:
yaml2json << EOI > runtime-actus-pam.actus
scheduleConfig:
  businessDayConvention: "NULL"
  endOfMonthConvention: "EOM"
  calendar: "NC"
maturityDate: "$MATURITY_DATE"
contractId: "0"
enableSettlement: false
initialExchangeDate: "$INITIAL_EXCHANGE_DATE"
contractRole: "RPA"
penaltyType: "O"
cycleAnchorDateOfInterestPayment: "$INITIAL_EXCHANGE_DATE"
contractType: "PAM"
notionalPrincipal: $PRINCIPAL
contractPerformance: "PF"
collateralAmount: 0
dayCountConvention: "30E360"
accruedInterest: 0
statusDate: "$STATUS_DATE"
cycleOfInterestPayment: "P1YL1"
prepaymentEffect: "N"
nominalInterestRate: $INTEREST_RATE
interestCalculationBase: "NT"
EOI
cat runtime-actus-pam.actus

{"accruedInterest":0,"collateralAmount":0,"contractId":"0","contractPerformance":"PF","contractRole":"RPA","contractType":"PAM","cycleAnchorDateOfInterestPayment":"2023-01-01T00:00:00","cycleOfInterestPayment":"P1YL1","dayCountConvention":"30E360","enableSettlement":false,"initialExchangeDate":"2023-01-01T00:00:00","interestCalculationBase":"NT","maturityDate":"2024-01-01T00:00:00","nominalInterestRate":0.02,"notionalPrincipal":100,"penaltyType":"O","prepaymentEffect":"N","scheduleConfig":{"businessDayConvention":"NULL","calendar":"NC","endOfMonthConvention":"EOM"},"statusDate":"2022-10-10T00:00:00"}


Create the contract and its initial state.

In [20]:
marlowe-cli template actus --minimum-ada "$MINIMUM_ADA" \
                           --party "$PARTY_ADDR" \
                           --counter-party "$COUNTERPARTY_ADDR" \
                           --actus-terms-file runtime-actus-pam.actus \
                           --out-contract-file runtime-actus-pam-1.contract \
                           --out-state-file    runtime-actus-pam-1.state

View the contract.

In [21]:
json2yaml runtime-actus-pam-1.contract

timeout: 1672531200000
timeout_continuation: close
when:
- case:
    deposits:
      negate:
        negate: 100000000
    into_account:
      address: addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q
  then:
    from_account:
      address: addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q
    pay: 100000000
    then:
      timeout: 1704067200000
      timeout_continuation: close
      when:
      - case:
          deposits: 2000000
          into_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
        then:
          from_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdm

View the initial state.

In [22]:
json2yaml runtime-actus-pam-1.state

accounts:
- - - address: addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q
    - currency_symbol: ''
      token_name: ''
  - 3000000
boundValues: []
choices: []
minTime: 1


## Create two transactions so that they can be viewed by Marlowe History

### Transaction 1. Create the ontract

First we create a `.marlowe` file that contains the initial information needed to run the contract.

In [23]:
marlowe-cli run initialize --contract-file runtime-actus-pam-1.contract \
                           --state-file    runtime-actus-pam-1.state    \
                           --out-file      runtime-actus-pam-1.marlowe  \
                           --print-stats


Validator size: 12505
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}


Submit the transaction.

In [24]:
TX_1=$(
marlowe-cli run auto-execute --marlowe-out-file runtime-actus-pam-1.marlowe \
                             --required-signer "$PARTY_SKEY" \
                             --change-address "$PARTY_ADDR" \
                             --out-file runtime-actus-pam-1.raw \
                             --print-stats \
                             --submit 600 \
)
echo "TX_1 = $TX_1"


Fee: Lovelace 220457
Size: 1133 / 16384 = 6%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
TX_1 = 50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4


The contract is identified by its first UTxO.

In [25]:
CONTRACT_ID="$TX_1#1"
echo "$CONTRACT_ID"

50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4#1


### Transaction 2. Party deposits loan amount

In [26]:
marlowe-cli run prepare --marlowe-file runtime-actus-pam-1.marlowe \
                        --out-file     runtime-actus-pam-2.marlowe \
                        --deposit-account "$PARTY_ADDR" \
                        --deposit-party "$PARTY_ADDR" \
                        --deposit-amount "$((PRINCIPAL*FIXED_POINT))" \
                        --invalid-before "$((NOW - 5 * MINUTE))" \
                        --invalid-hereafter "$((NOW + 1 * HOUR))" \
                        --print-stats


Datum size: 605
Payment 1
  Acccount: "\"addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q\""
  Payee: Party "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\""
  Ada: Lovelace {getLovelace = 100000000}


Submit the transaction.

In [27]:
TX_2=$(
marlowe-cli run auto-execute --tx-in-marlowe "$TX_1#1" \
                             --marlowe-in-file  runtime-actus-pam-1.marlowe \
                             --marlowe-out-file runtime-actus-pam-2.marlowe \
                             --required-signer "$PARTY_SKEY" \
                             --change-address "$PARTY_ADDR" \
                             --out-file runtime-actus-pam-2.raw \
                             --print-stats \
                             --submit 600 \
)
echo "TX_2 = $TX_2"


Fee: Lovelace 1291586
Size: 14464 / 16384 = 88%
Execution units:
  Memory: 6247626 / 14000000 = 44%
  Steps: 1719670345 / 10000000000 = 17%
TX_2 = b1736fdd2f5ed856ad85e1cfdac4802583484a9cbe0752f8912ca76080385e5f


See if we could have submitted the transaction via Marlowe Runtime.

In [28]:
marlowe submit runtime-actus-pam-2.raw

marlowe: not implemented
CallStack (from HasCallStack):
  error, called at cli/Language/Marlowe/Runtime/CLI/Command/Submit.hs:22:20 in main:Language.Marlowe.Runtime.CLI.Command.Submit


: 1

<span style="color: red; font-weight: bold">It looks like this hasn't been implemented yet.</span>

## Contract history commands

### Add the contract to Marlowe History

In [29]:
marlowe add "$CONTRACT_ID"

50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4#1


### List the contracts managed by Marlowe History

In [30]:
marlowe ls --show-status

50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4#1 Status: Waiting (SomeMarloweVersion MarloweV1)


### Check removal of the contract from Marlowe History

Remove it.

In [31]:
marlowe rm "$CONTRACT_ID"

50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4#1


See that it has been removed.

In [32]:
marlowe ls

Add it back.

In [33]:
marlowe add "$CONTRACT_ID"

50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4#1


In [34]:
marlowe ls

50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4#1


### List all Marlowe contracts

In [35]:
marlowe ls --all --show-status | head -n 20

02811e36c6cdac4721b53f718c4a1406e09ef0d985f9ad6b7fd676769e2f866c#1 Status: not added
045d4ac6f50c57cade58fd697b907ff0418637ba1eadb36392e04d1c5d489b21#1 Status: not added
0552fc00860d35d45c09b4b43740193579adbeea5097cf33b62201332e3095ee#1 Status: not added
05574d9d1935d2a5d2e3cb15e79d46b076e4c81d85bd2e12940ed47b628001be#1 Status: not added
05990678496287262051fc831ff35903e383ffc564b943de76e78394a8ceb4cd#1 Status: not added
061d29872aae0580496aecc8bed3a7a11d743c55d24f6d2c5c03c630cf89e899#1 Status: not added
066727dc5d40a6df33bb88dea3d29a9c1bb9b68493870ef9a04d6cbffe11e21b#1 Status: not added
06718671646f66d1818fce1d2532a3bb2050a42df164c71c3306f9c67ba6f1ab#1 Status: not added
06ddf0488df24a26882abf9546a8d314994b768465246172f8b4492e8a820162#1 Status: not added
072b7513bfae663c993d21977e6b1c2231a6fb389d09595a1e3d4ee517b5a6ed#1 Status: not added
07475c7a0206e0ec98e52c6891bfa5cda2e65b814f08f66f19a7d9958af989ad#1 Status: not added
0b25d59061a5748f0aef14b5a4e8829f0a4940a7aa988325bfd197584de2da58#

### See if there are any failed contracts

In [36]:
marlowe ls --all > all.tmp
marlowe ls --all --show-failed > failed.tmp
diff all.tmp failed.tmp

<span style="color: red; font-weight: bold">I think that <code>--show-failed</code> should <em>only</em> show the failed contracts, not all of them.</span>

### View the basic information about the contract

In [37]:
marlowe log "$CONTRACT_ID"

transaction 50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4 (creation)
ContractId:      50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4#1
SlotNo:          5430824
BlockNo:         251684
BlockId:         d7bad347dbd8e8f3e676b010d2432b2b8606b006350730068226f8c27fccc236
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1



transaction b1736fdd2f5ed856ad85e1cfdac4802583484a9cbe0752f8912ca76080385e5f
ContractId: 50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4#1
SlotNo:     5430833
BlockNo:    251685
BlockId:    c06bfa8d6566f215672776b2d656cf6261a69e328ff3a7c1639dfd82bfee3c3d
Inputs:     [Input (IDeposit "\"addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q\"" "\"addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q\"" (Token "" "") 100000000)]




<span style="color: red; font-weight: bold">I think that <code>marlowe log</code> should print something parseable like YAML.</span>

### View the contract itself

In [38]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4 (creation)
ContractId:      50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4#1
SlotNo:          5430824
BlockNo:         251684
BlockId:         d7bad347dbd8e8f3e676b010d2432b2b8606b006350730068226f8c27fccc236
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q" Address "addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

<span style="color: red; font-weight: bold">I think that <code>marlowe log --show-contract</code> should print something parseable like YAML.</span>

## Finish running the contract

### Transaction 3. Counterparty repays the loan's interest

In [39]:
marlowe-cli run prepare --marlowe-file runtime-actus-pam-2.marlowe \
                        --out-file     runtime-actus-pam-3.marlowe \
                        --deposit-account "$COUNTERPARTY_ADDR" \
                        --deposit-party "$COUNTERPARTY_ADDR" \
                        --deposit-amount "$((INTEREST*FIXED_POINT))" \
                        --invalid-before "$((NOW-5*MINUTE))" \
                        --invalid-hereafter "$((NOW+1*HOUR))" \
                        --print-stats


Datum size: 349
Payment 1
  Acccount: "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\""
  Payee: Party "\"addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q\""
  Ada: Lovelace {getLovelace = 2000000}


In [40]:
TX_3=$(
marlowe-cli run auto-execute --tx-in-marlowe "$TX_2#1" \
                             --marlowe-in-file  runtime-actus-pam-2.marlowe \
                             --marlowe-out-file runtime-actus-pam-3.marlowe \
                             --required-signer "$COUNTERPARTY_SKEY" \
                             --change-address "$COUNTERPARTY_ADDR" \
                             --out-file runtime-actus-pam-3.raw \
                             --print-stats \
                             --submit 600 \
)
echo "TX_3 = $TX_3"


Fee: Lovelace 1233027
Size: 13952 / 16384 = 85%
Execution units:
  Memory: 5815658 / 14000000 = 41%
  Steps: 1565634381 / 10000000000 = 15%
TX_3 = 38bff22b6eb7b60ce62ece1a967db6533a578d4a0e139baf5bb7a8a2b12240d5


### Transaction 4. Counterparty repays the loan's principal

In [41]:
marlowe-cli run prepare --marlowe-file runtime-actus-pam-3.marlowe \
                        --out-file     runtime-actus-pam-4.marlowe \
                        --deposit-account "$COUNTERPARTY_ADDR" \
                        --deposit-party "$COUNTERPARTY_ADDR" \
                        --deposit-amount "$((PRINCIPAL*FIXED_POINT))" \
                        --invalid-before "$((NOW-5*MINUTE))" \
                        --invalid-hereafter "$((NOW+4*HOUR))" \
                        --print-stats


Datum size: 30
Payment 1
  Acccount: "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\""
  Payee: Party "\"addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q\""
  Ada: Lovelace {getLovelace = 100000000}
Payment 2
  Acccount: "\"addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q\""
  Payee: Party "\"addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q\""
  Ada: Lovelace {getLovelace = 3000000}


In [42]:
TX_4=$(
marlowe-cli run auto-execute --tx-in-marlowe "$TX_3#1" \
                             --marlowe-in-file  runtime-actus-pam-3.marlowe \
                             --marlowe-out-file runtime-actus-pam-4.marlowe \
                             --required-signer "$COUNTERPARTY_SKEY" \
                             --change-address "$COUNTERPARTY_ADDR" \
                             --out-file runtime-actus-pam-4.raw \
                             --print-stats \
                             --submit 600 \
)
echo "TX_4 = $TX_4"


Fee: Lovelace 1078968
Size: 13273 / 16384 = 81%
Execution units:
  Memory: 4210566 / 14000000 = 30%
  Steps: 1127777514 / 10000000000 = 11%
TX_4 = 157a1de93effb91ea1ddf11d9d6fdc73db3dcb014dc42ef588c8cb6cf3f93a79


The contract has closed.

In [43]:
jq '.tx.contract' runtime-actus-pam-4.marlowe | json2yaml

close
...


<span style="color: red; font-weight: bold">The contract has closed, but it looks like Marlowe History thinks it is still open:</span>

In [44]:
marlowe ls --show-status

50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4#1 Status: Waiting (SomeMarloweVersion MarloweV1)


In [45]:
marlowe log "$CONTRACT_ID"

transaction 50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4 (creation)
ContractId:      50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4#1
SlotNo:          5430824
BlockNo:         251684
BlockId:         d7bad347dbd8e8f3e676b010d2432b2b8606b006350730068226f8c27fccc236
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1



transaction b1736fdd2f5ed856ad85e1cfdac4802583484a9cbe0752f8912ca76080385e5f
ContractId: 50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4#1
SlotNo:     5430833
BlockNo:    251685
BlockId:    c06bfa8d6566f215672776b2d656cf6261a69e328ff3a7c1639dfd82bfee3c3d
Inputs:     [Input (IDeposit "\"addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q\"" "\"addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q\"" (Token "" "") 100000000)]


transaction 38bff22b6eb7b60ce62ece1a967db6533a578d4a0e139baf5bb7a8a2b12240d5
ContractId: 50e60c9c2fda6261ae92b5d7ffd80d9b268e

In [46]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4 (creation)
ContractId:      50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4#1
SlotNo:          5430824
BlockNo:         251684
BlockId:         d7bad347dbd8e8f3e676b010d2432b2b8606b006350730068226f8c27fccc236
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q" Address "addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vzz3vhpuz09q4q4zx9efhwu04jfy5rv54cc3u2wuahxkf2qvxzf6q"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

## Additional testing

<span style="color: red; font-weight: bold">We still have other commands to test:</span>

* apply
* advance
* deposit
* choose
* notify
* create
* withdraw

In [47]:
marlowe --help

Usage: marlowe [--history-host HOST_NAME] [--history-command-port PORT_NUMBER] 
               [--history-query-port PORT_NUMBER] 
               [--history-sync-port PORT_NUMBER] [--discovery-host HOST_NAME] 
               [--discovery-query-port PORT_NUMBER] [--tx-host HOST_NAME] 
               [--tx-command-port PORT_NUMBER] (COMMAND | COMMAND | COMMAND)

  Command line interface for managing Marlowe smart contracts on Cardano.

Available options:
  -h,--help                Show this help text
  --history-host HOST_NAME The hostname of the Marlowe Runtime history server.
                           Can be set as the environment variable
                           MARLOWE_RT_HISTORY_HOST (default: "127.0.0.1")
  --history-command-port PORT_NUMBER
                           The port number of the history server's job API. Can
                           be set as the environment variable
                           MARLOWE_RT_HISTORY_COMMAND_PORT (default: 23717)
  --history-query-port

In [48]:
marlowe rm "$CONTRACT_ID"

50e60c9c2fda6261ae92b5d7ffd80d9b268eee144d7d4a1e137976f46edc41e4#1


In [49]:
marlowe ls